# Generate daily bars using daily data service - C#

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.<br>

This sample demonstrates how to directly request and plot *`daily bars`*.<br>

It enables to retrieve **on-demand** daily bars data by calling a dedicated service on a **daily** data store. 

### Inputs/outputs
Bars sample requires instrument's identifier as per input. It returns open-high-low-close (OHLC) and volume.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyBarsService*: to directly retrieve daily bars objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily bars sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Daily.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily bars* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyBarsService* in order to retrieve daily data

In [ ]:
// Set instrument
var ticker = "AAPL";
var exchange = "XNGS";

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Instantiate the daily bars service
var service = new DailyBarsService.DailyBarsServiceClient(channel);

// Create and process daily bars request
var request = new DailyBarsRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}
};
var reply = service.DailyBars(request, headers);

The following code snippet displays the *daily bars* data points:

In [ ]:
display(reply.Data)

### Step 4: Visualize data

#### 4.1 Plot the closing price and volume
The following code snippets retrive *Close prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package. Plot a simple line graph with the *close price* and *volume* to easily spot spikes in trading as follows: 

In [ ]:
// Extract the close price and the volume
var closePrices = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Close));
var volumes = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Volume));

In [ ]:
// Create the graph
var closesGraph = new List<Scattergl>
{
    new Scattergl
    {
        name = "Close Price",
        mode = "lines",
        x = closePrices.Select(c => c.Item1),
        y = closePrices.Select(c => c.Item2)
    },
    new Scattergl
    {
        name = "Volume",
        mode = "bar",
        x = volumes.Select(v => v.Item1),
        y = volumes.Select(v => v.Item2),
        yaxis = "y2"
    } 
};


var layout = new Layout.Layout{ 
    title = $"Close price & Volume for {ticker}",
    yaxis = new Yaxis {
        title = "Price", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Volume", 
        autorange = true,
        side = "right",  
    },
};

var chart = Chart.Plot(closesGraph, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);

#### 4.2 Plot the bars chart
To generate the bars graph we need to pass Open, High, Low and Close prices as per parameters:

In [ ]:
var bars = reply.Data.Select(b => new Tuple<DateTime,double,double,double,double>(new DateTime(b.Date.Year, b.Date.Month, b.Date.Day), b.Open, b.High, b.Low, b.Close));

In [ ]:
var chart = Chart.Candlestick(bars);
chart.WithTitle($"Bars chart for {ticker}");
chart.WithXTitle("Time");
chart.WithYTitle("OHLC");
chart.Width = 1500;
chart.Height = 800;
display(chart);